#**Step 1: installazione delle librerie**

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


#**Step 2: caricamento delle librerie**

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import pandas as pd
from datasets import Dataset
import re
import random

import gc
import warnings
import ast


In [3]:
warnings.filterwarnings("ignore", category=UserWarning, message="torch.utils.checkpoint")
warnings.filterwarnings("ignore", category=UserWarning, message="None of the inputs have requires_grad=True")

#**Step 3: collegamento a drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# change this to match your path
data_path_models = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 3a/Models"
data_path_data = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 3a/Datasets"

Mounted at /content/drive


#**Step 3: impostazione delle variabili**

In [8]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
fine_tuned_model_name = "llama2_7b_it_2epoch"
testset_name = "splitted_test"

# Load the entire model on the GPU 0
device_map = {"": 0}

#**Step 4: generazione**

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

def generate_text_for_sentence_aggregation(triple, sentence_len):
  max_length = len(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] [/ANW]") + sentence_len * 1.2

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=int(max_length), temperature=0.1)
  result = pipe(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE]")

  if result is not None and result and len(result) > 0 and 'generated_text' in result[0]:
      generated_text = result[0]['generated_text']

      # Utilizziamo una regex per estrarre il contenuto all'interno del primo tag [ANW] ... [/ANW]
      match = re.search(r'\[ANW\] (.*?) \[/ANW\]', generated_text)

      if match:
          content_within_anw_tag = match.group(1)
          text = content_within_anw_tag
      else:
          text = generated_text

  else:
      text = "Il risultato della pipeline è nullo."

  return text

def generation_for_sentence_aggregation(testset, testset_name):
  current_id = -1
  rows = []
  row = {}
  for i in range(len(testset)):
    if (testset['id'][i] < 1010):
      id = testset['id'][i]
      triples = testset['triples'][i]
      data_unit = testset['data_unit'][i]
      shape = testset['shape'][i]
      shape_type = testset['shape_type'][i]
      local_size = testset['local_size'][i]
      size = testset['size'][i]

      if current_id == -1 or current_id != id:
        if current_id != -1:
          rows.append(row)

        row = {
            'id' : id,
            'data_unit' : data_unit,
            'sentences': [],
            'shape': shape,
            'shape_type': shape_type,
            'size': size,
        }

      prediction = generate_text_for_sentence_aggregation(triples, len(triples))
      row['sentences'].append(prediction)
      current_id = id

      print(f"======== Step {i} ========")
      print(f"tripla: {triples}")
      print(f"predizione: {prediction}")
      print('\n')

      # Crea un DataFrame da rows
      df = pd.DataFrame(rows)
      #df.to_csv(f'{data_path_gem_models}/{new_model}_decoding_to_aggregate.csv', index=False)
      df.to_csv(f'{data_path_models}/{testset_name}_step1.csv', index=False)

  # Crea un DataFrame da rows
  df = pd.DataFrame(rows)
  #df.to_csv(f'{data_path_gem_models}/{new_model}_decoding_to_aggregate.csv', index=False)
  df.to_csv(f'{data_path_models}/{testset_name}_step1.csv', index=False)


In [ ]:
# caricamento di llama fine tuned
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, f'{data_path_models}/{fine_tuned_model_name}')
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# caricamento del testset
testset = pd.read_csv(f'{data_path_data}/{testset_name}.csv')
testset.head()

In [ ]:
# generazione delle frasi
generation_for_sentence_aggregation(testset, testset_name)

#**Step 4: Sentence Aggregation**

In [20]:
def generate_sentence_aggregation2(sentences):
  #text = "instruction=\"Your task is to paraphrase and put together following the order I give you the following texts and do *not* add any extra information. You have to generate the sentence in Italian.\""
  text = "instruction=\"You have to aggregate and paraphrase together the following sentences. You have to generate the result in Italian."
  text += "For example, given the sentences \"Il Monumento alla Battaglia delle Nazioni si trova al 200 di Public Square.\" and"
  text += "\"Il Monumento alla Battaglia delle Nazioni è stato progettato da Hugo Lloris\""
  text += " generete a sentence like this \"Il Monumento alla Battaglia delle Nazioni, progettato da Hugo Lloris, si trova al 200 di Public Square\"\""

  text += " Input:"
  text += " texts="
  text += " " + sentences
  text += " Output:"

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=len(text), temperature=0.1)
  result = pipe(f"{text}")
  text = result[0]['generated_text']

  match = re.search(r'Output: "(.*?)"', text)
  if match:
    prediction = match.group(1)
  else:
    prediction = text

  return prediction

def generate_sentence_aggregation(sentences):
  # PROMPT 1
  #text = "### Instruction: {Your task is put together the following texts and paraphrase to get more fluency. Do *not* add any extra information and generate the sentence in Italian."
  #text = "For example, given ""}"
  #text += " ### Input: {" + sentences + "}"
  #text += " ### Response:"


  instruction = "Your task is put together the following texts and paraphrase to get more fluency. Do *not* add any extra information and generate the sentence in Italian."

  text = f"<s>[INST]<<SYS>>{instruction}<</SYS>>{sentences}[/INST]"

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=len(text), temperature=0.1)
  result = pipe(f"{text}")
  text = result[0]['generated_text']

  match = re.search(r'Response: (.*)', text, re.DOTALL)
  if match:
    prediction = match.group(1)
  else:
    prediction = text

  return prediction


def sentence_aggregation(testset, testset_name):
  rows = []
  for i in range(len(testset)):
    sentences = ast.literal_eval(testset['sentences'][i])
    sentences_text = ''
    j = 1

    if len(sentences) > 1:
      for sentence in sentences:
        sentences_text += f"Text {j}: \"{sentence}\"\n"
        j = j + 1
      prediction = generate_sentence_aggregation2(sentences_text)
    else:
      sentences_text += f"Text {j}: \"{sentences[0]}\"\n"
      prediction = " " . join(sentences)

    print(f"===== STEP {i}")
    print(f"sentences_text: {sentences_text}")
    print(f"prediction: {prediction}")
    print('\n')

    row = {
        'data_unit' : testset['data_unit'][i],
        'predictions' : prediction,
    }

    rows.append(row)

    df = pd.DataFrame(rows)
    df.to_csv(f'{data_path_models}/{testset_name}_step2.csv', index=False)

  df = pd.DataFrame(rows)
  df.to_csv(f'{data_path_models}/{testset_name}_step2.csv', index=False)


In [ ]:
# pulizia
del base_model
del model
del tokenizer
gc.collect()
gc.collect()

0

In [ ]:
gc.collect()
gc.collect()

0

In [9]:
# caricamento di llama
bnb_config = BitsAndBytesConfig(
  load_in_4bit= True,
  bnb_4bit_quant_type= "nf4",
  bnb_4bit_compute_dtype= torch.bfloat16,
  bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

(True, True)

In [10]:
# caricamento del test set
testset = pd.read_csv(f'{data_path_models}/{testset_name}_step1.csv')
testset.head()

,id,data_unit,sentences,shape,shape_type,size
0,0,11th_Mississippi_Infantry_Monument category Co...,"[""Il Monumento all'11° Fanteria del Mississipp...",(X (X) (X) (X (X))),mixed,4
1,1,Bananaman broadcastedBy BBC | BBC city Broadca...,['Bananaman è stato trasmesso dalla BBC e inte...,(X (X) (X (X))),mixed,3
2,2,Karlsruhe postalCode 76131–76229 |,['Il codice postale di Karlsruhe è 76131-76229.'],(X (X)),NaN,1
3,3,Turkish_people religion Irreligion |,"[""La religione dei turchi è l'irreligione.""]",(X (X)),NaN,1
4,4,ENAIRE city Madrid | Adolfo_Suárez_Madrid–Bara...,"['ENAIRE si trova nella città di Madrid.', ""L'...",(X (X) (X) (X) (X (X))),mixed,5


In [21]:
# generazione delle frasi
sentence_aggregation(testset, f'{testset_name}_prompt3')

Output streaming troncato alle ultime 5000 righe.

===== STEP 265
sentences_text: Text 1: "Grigory Neujmin è morto il 17 dicembre 1946."

prediction: Grigory Neujmin è morto il 17 dicembre 1946.


===== STEP 266
sentences_text: Text 1: "1147 Stavropolis, scoperta il 1929-06-11, ha una magnitudine assoluta di 11,5 e un apoapside di 418476000000.0."
Text 2: "1147 Stavropolis è stato scoperto da Grigory Neujmin e ha una data epocale del 31 luglio 2016 (JD2457600.5) e un periodo orbitale di 1249,6."
Text 3: "1147 Stavropolis ha un periodo di rotazione di 20378,5."

prediction: 1147 Stavropolis, scoperto da Grigory Neujmin, ha una magnitudine assoluta di 11,5 e un apoapside di 41847600000.0, è stato scoperto il 1929-06-11 e ha una data epocale del 31 luglio 2016 (JD2457600.5), ha un periodo orbitale di 1249,6 e un periodo di rotazione di 20378,5.


===== STEP 267
sentences_text: Text 1: "Ciudad Ayala, in Messico, si trova a 1147 metri sul livello del mare e il suo leader è il Governatore."
